In [14]:
import random
import math
import statistics

In [15]:
# Função de Parsing
def parse_evrp_instance(filepath):
    """
    Extraindo informações de uma instância
    """
    with open(filepath, 'r') as f:
        lines = f.readlines()

    dimension = 0
    stations_count = 0
    num_vehicles = 0 
    
    node_coords_section = False
    demand_section = False
    stations_coord_section = False
    reading_depot_id = False 
    
    coordenadas = {}
    demandas = {} 
    depot_id = -1
    
    for line_idx, raw_line in enumerate(lines):
        line = raw_line.strip()
        if not line:
            continue

        if line.startswith("DIMENSION:"):
            dimension = int(line.split(":")[1].strip())
        elif line.startswith("STATIONS:"):
            stations_count = int(line.split(":")[1].strip())
        elif line.startswith("VEHICLES:"): 
            num_vehicles = int(line.split(":")[1].strip())
        elif line.startswith("NODE_COORD_SECTION"):
            node_coords_section = True
            demand_section = False
            stations_coord_section = False
            reading_depot_id = False 
            continue
        elif line.startswith("DEMAND_SECTION"):
            demand_section = True
            node_coords_section = False
            stations_coord_section = False
            reading_depot_id = False 
            continue
        elif line.startswith("STATIONS_COORD_SECTION"):
            stations_coord_section = True
            node_coords_section = False
            demand_section = False
            reading_depot_id = False 
            continue
        elif line.startswith("DEPOT_SECTION"):
            reading_depot_id = True 
            node_coords_section = False
            demand_section = False
            stations_coord_section = False
            continue
        elif line.startswith("EOF"):
            break

        if reading_depot_id:
            try:
                depot_id = int(line)
                reading_depot_id = False
                if depot_id == -1: 
                    break 
            except ValueError:
                print(f"Aviso: Esperado ID do depósito numérico, encontrado '{line}' após DEPOT_SECTION.")
                reading_depot_id = False
            continue 

        if node_coords_section and line[0].isdigit():
            parts = line.split()
            if len(parts) >= 3: 
                node_id = int(parts[0])
                x = float(parts[1])
                y = float(parts[2])
                coordenadas[node_id] = (x, y)

        elif demand_section and line[0].isdigit():
            parts = line.split()
            if len(parts) >= 2: 
                node_id = int(parts[0])
                demand = int(parts[1]) 
                demandas[node_id] = demand
        
        elif stations_coord_section and line[0].isdigit():
            pass

    client_nodes = [node_id for node_id, demand in demandas.items() if demand > 0]
    
    return dimension, stations_count, num_vehicles, coordenadas, demandas, depot_id, client_nodes

In [16]:
def calcular_matriz_distancias(coordenadas):
    """Calcula a matriz de distâncias euclidianas entre todos os nós."""
    distancias = {}
    for i in coordenadas:
        distancias[i] = {}
        for j in coordenadas:
            if i == j:
                distancias[i][j] = 0
            else:
                x1, y1 = coordenadas[i]
                x2, y2 = coordenadas[j]
                distancias[i][j] = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    return distancias

In [17]:
# Criação de Indivíduo (Permutação)

def criar_individuo_permutacao(client_nodes_list):
    """
    Cria um indivíduo representado por uma permutação aleatória dos IDs dos clientes.
    """
    permutacao = list(client_nodes_list) 
    random.shuffle(permutacao) 
    return permutacao

def criar_populacao_inicial_permutacao(tamanho_populacao, client_nodes_list):
    """
    Cria a população inicial, onde cada indivíduo é uma permutação aleatória de clientes.
    """
    return [criar_individuo_permutacao(client_nodes_list) for _ in range(tamanho_populacao)]

In [18]:
# Funções de Operadores Genéticos para Permutação ---

def cycle_crossover(parent1_perm, parent2_perm):
    """
    Cycle Crossover (CX) para permutações.
    Gera dois filhos a partir de dois pais.
    """
    size = len(parent1_perm)
    child1_perm = [-1] * size
    child2_perm = [-1] * size
    
    visited = [False] * size
    
    # Gerando child 1
    cycle_start_idx = 0
    while cycle_start_idx < size:
        if not visited[cycle_start_idx]:
            # Inicia um novo ciclo
            current_idx = cycle_start_idx
            first_element_of_cycle = parent1_perm[current_idx]
            
            # Copia elementos de parent1 deste ciclo para child1
            # And from parent2 to child2
            while True:
                child1_perm[current_idx] = parent1_perm[current_idx]
                child2_perm[current_idx] = parent2_perm[current_idx]
                visited[current_idx] = True
                
                # Encontra a posição do elemento de parent1_perm em parent2_perm
                current_element_in_p1 = parent1_perm[current_idx]
                next_idx = parent2_perm.index(current_element_in_p1) # Encontra onde este elemento está em parent2
                
                if next_idx == cycle_start_idx:
                    break # Ciclo completo
                current_idx = next_idx
        
        # Mover para a próxima posição não visitada para iniciar um novo ciclo
        cycle_start_idx += 1
        while cycle_start_idx < size and visited[cycle_start_idx]:
            cycle_start_idx += 1

    # Preencher posições restantes: troca os elementos child1 e child2 baseado no status de visita
    for i in range(size):
        if child1_perm[i] == -1: # Se não preenchido por um ciclo, significa que era parte de um ciclo par
            child1_perm[i] = parent2_perm[i]
            child2_perm[i] = parent1_perm[i]
    
    return child1_perm, child2_perm

In [19]:
def inversion_mutation(permutation):
    """
    Mutação por Inversão: inverte uma sub-sequência da permutação.
    """
    if len(permutation) < 2:
        return permutation
        
    idx1, idx2 = random.sample(range(len(permutation)), 2)
    if idx1 > idx2:
        idx1, idx2 = idx2, idx1
    
    # Inverte a sub-lista entre idx1 e idx2 (inclusive)
    # Ex: [1,2,3,4,5], idx1=1, idx2=3 -> [1, 4,3,2, 5]
    inverted_segment = permutation[idx1 : idx2 + 1]
    inverted_segment.reverse()
    
    new_permutation = permutation[:idx1] + inverted_segment + permutation[idx2 + 1:]
    
    return new_permutation

In [20]:
# Função de Aptidão (Fitness) para VRP Baseado em Permutação - SEM RESTRIÇÕES

avaliacoes_count = 0 

def calcular_aptidao_vrp_permutacao(individuo_permutacao, distancias_map, depot_node_id, num_vehicles):
    """
    Calcula a aptidão de um indivíduo (permutação de clientes) para o VRP,
    distribuindo os clientes entre o número de veículos disponíveis.
    A aptidão é o inverso da distância total.
    NÃO considera capacidade de carga ou bateria.
    """
    global avaliacoes_count
    avaliacoes_count += 1

    num_clientes = len(individuo_permutacao)
    
    if num_clientes == 0:
        return float('inf') 

    segment_size = math.ceil(num_clientes / num_vehicles) 
    
    total_distance = 0
    
    current_client_idx = 0

    for _ in range(num_vehicles):
        if current_client_idx >= num_clientes:
            break 
        
        route_segment_end_idx = min(current_client_idx + segment_size, num_clientes)
        route_clients = individuo_permutacao[current_client_idx : route_segment_end_idx]
        
        if route_clients: 
            route = [depot_node_id] + route_clients + [depot_node_id]
            for k in range(len(route) - 1):
                total_distance += distancias_map[route[k]][route[k+1]]

        current_client_idx = route_segment_end_idx
    
    if total_distance == 0:
        return float('inf') 
    
    return 1 / total_distance

In [21]:
# Seleção por Torneio

def selecionar_pais_torneio(populacao, aptidoes, tamanho_torneio):
    """
    Seleciona dois pais da população usando seleção por torneio.
    """
    
    def selecionar_um_pai(pop, apts, k):
        """Seleciona um indivíduo de um torneio de tamanho k."""
        torneio_indices = random.sample(range(len(pop)), k)
        melhor_individuo = None
        melhor_aptidao_torneio = -1

        for idx in torneio_indices:
            if apts[idx] > melhor_aptidao_torneio:
                melhor_aptidao_torneio = apts[idx]
                melhor_individuo = pop[idx]
        return melhor_individuo

    pai1 = selecionar_um_pai(populacao, aptidoes, tamanho_torneio)
    pai2 = selecionar_um_pai(populacao, aptidoes, tamanho_torneio)
    
    return pai1, pai2

In [22]:
# Função Principal do Algoritmo Genético (Permutação com Inversion, Cycle Crossover, Torneio)

def algoritmo_genetico_vrp_permutacao_avancado(
    tamanho_populacao, 
    taxa_mutacao, 
    max_avaliacoes,
    distancias_map,
    depot_node_id,
    client_nodes_list,
    num_vehicles,
    tamanho_torneio      
):
    global avaliacoes_count
    avaliacoes_count = 0 

    populacao = criar_populacao_inicial_permutacao(tamanho_populacao, client_nodes_list)
    
    melhor_solucao_permutacao = None 
    melhor_aptidao = -1
    
    geracao = 0
    while avaliacoes_count < max_avaliacoes:
        aptidoes = []
        for ind in populacao:
            apt = calcular_aptidao_vrp_permutacao(ind, distancias_map, depot_node_id, num_vehicles)
            aptidoes.append(apt)
            if apt > melhor_aptidao:
                melhor_aptidao = apt
                melhor_solucao_permutacao = list(ind) 

        nova_populacao = []
        if melhor_solucao_permutacao:
            nova_populacao.append(list(melhor_solucao_permutacao)) 

        while len(nova_populacao) < tamanho_populacao and avaliacoes_count < max_avaliacoes:
            pai1, pai2 = selecionar_pais_torneio(populacao, aptidoes, tamanho_torneio) 
            
            # USO DE CYCLE CROSSOVER
            filho1_perm, filho2_perm = cycle_crossover(list(pai1), list(pai2)) 
            
            # USO DE INVERSION MUTATION
            if random.random() < taxa_mutacao:
                mutacao_filho1 = inversion_mutation(list(filho1_perm))
            else:
                mutacao_filho1 = list(filho1_perm) 
            
            if random.random() < taxa_mutacao:
                mutacao_filho2 = inversion_mutation(list(filho2_perm))
            else:
                mutacao_filho2 = list(filho2_perm) 
            
            nova_populacao.append(mutacao_filho1)
            if len(nova_populacao) < tamanho_populacao:
                nova_populacao.append(mutacao_filho2)
        
        populacao = nova_populacao
        geracao += 1
        
    distancia_minima = 1 / melhor_aptidao if melhor_aptidao != float('inf') and melhor_aptidao > 0 else float('inf')
    
    rotas_finais = []
    if melhor_solucao_permutacao:
        num_clientes_final_solucao = len(melhor_solucao_permutacao)
        segment_size = math.ceil(num_clientes_final_solucao / num_vehicles)
        current_client_idx = 0

        for _ in range(num_vehicles):
            if current_client_idx >= num_clientes_final_solucao:
                break
            
            route_segment_end_idx = min(current_client_idx + segment_size, num_clientes_final_solucao)
            route_clients = melhor_solucao_permutacao[current_client_idx : route_segment_end_idx]
            
            if route_clients:
                route = [depot_node_id] + route_clients + [depot_node_id]
                rotas_finais.append(route)
            else:
                rotas_finais.append([depot_node_id, depot_node_id])

            current_client_idx = route_segment_end_idx

    return melhor_solucao_permutacao, distancia_minima, rotas_finais, avaliacoes_count

In [23]:
def estatisticas(minimo, maximo, media, stdev, opt_ref, mean_diff_percent, results, num_execucoes):
    # Extração e Apresentação das Estatísticas
    print("\n--- Estatísticas das Execuções do AG Avançado (Permutação) ---")
    print(f"Resultados das Distâncias ({num_execucoes} execuções):")
    
    for i, dist in enumerate(results):
        print(f"  Execução {i+1}: {dist:.2f}")

    print(f"\nMédia das Distâncias: {media:.2f}")
    print(f"Desvio Padrão das Distâncias: {stdev:.2f}")
    print(f"Valor Máximo da Distância: {maximo:.2f}")
    print(f"Valor Mínimo da Distância: {minimo:.2f}")
    print(f"\nValor Ótimo (do arquivo): {opt_ref:.2f}")
    print(f"Diferença percentual média em relação ao valor ótimo: {mean_diff_percent:.2f}%")

In [24]:
def escreve_arquivo(minimo, maximo, media, stdev, optimal_ref, mean_diff_percent, resultados_distancias_permutacao_avancado, NUM_EXECUCOES, FILE_PATH, num_vehicles_global, N_INSTANCIA, MAX_AVALIACOES_INSTANCIA, TAMANHO_POPULACAO, TAXA_MUTACAO, TAMANHO_TORNEIO, MASTER_SEED):
    # Bloco de Código para Escrever em Arquivo de Texto
    output_filename = FILE_PATH + ".txt"

    with open(output_filename, 'w') as f:
        f.write(f"--- Estatísticas das Execuções do AG Avançado (Permutação) ---\n")
        f.write(f"Instância: {FILE_PATH}\n")
        f.write(f"Semente Mestra do Experimento: {MASTER_SEED}\n") # Salva a semente mestra
        f.write(f"Número de Execuções: {NUM_EXECUCOES}\n")
        f.write(f"Número de Veículos: {num_vehicles_global}\n")
        f.write(f"Dimensão (N): {N_INSTANCIA}\n")
        f.write(f"Número máximo de avaliações por execução: {MAX_AVALIACOES_INSTANCIA}\n")
        f.write(f"Tamanho da População: {TAMANHO_POPULACAO}\n")
        f.write(f"Taxa de Mutação: {TAXA_MUTACAO}\n")
        f.write(f"Crossover: Cycle Crossover\n")
        f.write(f"Mutação: Inversion Mutation\n")
        f.write(f"Seleção: Torneio (k={TAMANHO_TORNEIO})\n")
        f.write(f"Representação: Permutação\n")
        f.write(f"Restrições de Capacidade/Bateria: NÃO CONSIDERADAS\n\n")

        f.write(f"Resultados das Distâncias Individuais:\n")
        for i, dist in enumerate(resultados_distancias_permutacao_avancado):
            f.write(f"  Execução {i+1}: {dist:.2f}\n")

        f.write(f"\n--- Resumo Estatístico ---\n")
        f.write(f"Média das Distâncias: {media:.2f}\n")
        f.write(f"Desvio Padrão das Distâncias: {stdev:.2f}\n")
        f.write(f"Valor Máximo da Distância: {maximo:.2f}\n")
        f.write(f"Valor Mínimo da Distância: {minimo:.2f}\n")
        f.write(f"\nValor Ótimo (do arquivo): {optimal_ref:.2f}\n")
        f.write(f"Diferença percentual média em relação ao valor ótimo: {mean_diff_percent:.2f}%\n")

    print(f"\nDados estatísticos salvos em '{output_filename}'")

In [25]:
def executar(instancia):
    # Bloco de Execução Principal

    # Carrega a instância
    FILE_PATH = instancia

    dimension, stations_count, num_vehicles_from_file, coordenadas_globais, demandas_globais, depot_id_global, client_nodes_global = parse_evrp_instance(
        FILE_PATH)

    NUM_CLIENTES_INSTANCIA = len(client_nodes_global)
    N_INSTANCIA = dimension
    distancias_instancia = calcular_matriz_distancias(coordenadas_globais)

    MAX_AVALIACOES_INSTANCIA = 25000 * N_INSTANCIA

    num_vehicles_global = 1

    # Parâmetros do AG baseado em Permutação
    TAMANHO_POPULACAO = 100
    TAXA_MUTACAO = 0.02
    NUM_EXECUCOES = 20

    TAMANHO_TORNEIO = 9

    MASTER_SEED = 21 # Semente mestra para replicar todo o experimento
    random.seed(MASTER_SEED)

    print(f"Iniciando {NUM_EXECUCOES} execuções do AG Avançado (Permutação) para a instância {FILE_PATH}")
    print(f"Semente Mestra do Experimento: {MASTER_SEED}") # Imprime a semente mestra
    print(f"Número de Veículos Disponíveis: {num_vehicles_global}")
    print(f"Dimensão (N): {N_INSTANCIA}")
    print(f"Número máximo de avaliações por execução: {MAX_AVALIACOES_INSTANCIA}")
    print(f"Tamanho da População: {TAMANHO_POPULACAO}")
    print(f"Taxa de Mutação: {TAXA_MUTACAO}")
    print(f"Crossover: Cycle Crossover")
    print(f"Mutação: Inversion Mutation")
    print(f"Seleção: Torneio (k={TAMANHO_TORNEIO})")
    print("\nAVISO: Este algoritmo NÃO considera capacidade de carga ou bateria nas rotas.")

    resultados_distancias_permutacao_avancado = []
    for i in range(NUM_EXECUCOES):
        print(f"\n--- Execução {i+1}/{NUM_EXECUCOES} ---")
        global avaliacoes_count
        avaliacoes_count = 0

        melhor_solucao, menor_distancia, rotas_finais, total_avaliacoes = algoritmo_genetico_vrp_permutacao_avancado(
            TAMANHO_POPULACAO,
            TAXA_MUTACAO,
            MAX_AVALIACOES_INSTANCIA,
            distancias_instancia,
            depot_id_global,
            client_nodes_global,
            num_vehicles_global,
            TAMANHO_TORNEIO
        )
        resultados_distancias_permutacao_avancado.append(menor_distancia)
        print(f"  Melhor solução: {melhor_solucao}")
        print(f"  Distância mínima encontrada nesta execução: {menor_distancia:.4f}")
        print(f"  Rotas finais: {rotas_finais}")
        print(f"  Total de avaliações nesta execução: {total_avaliacoes}")

    media = statistics.mean(resultados_distancias_permutacao_avancado)
    stdev = statistics.stdev(resultados_distancias_permutacao_avancado)
    maximo = max(resultados_distancias_permutacao_avancado)
    minimo = min(resultados_distancias_permutacao_avancado)

    if FILE_PATH == 'E-n23-k3.evrp':
        optimal_ref = 571.94
    elif FILE_PATH == 'E-n51-k5.evrp':
        optimal_ref = 529.90

    if optimal_ref > 0:
        mean_diff_percent = ((minimo - optimal_ref) / optimal_ref) * 100

    estatisticas(minimo, maximo, media, stdev, optimal_ref, mean_diff_percent,
                 resultados_distancias_permutacao_avancado, NUM_EXECUCOES)
    escreve_arquivo(minimo, maximo, media, stdev, optimal_ref, mean_diff_percent, resultados_distancias_permutacao_avancado, NUM_EXECUCOES,
                    FILE_PATH, num_vehicles_global, N_INSTANCIA, MAX_AVALIACOES_INSTANCIA, TAMANHO_POPULACAO, TAXA_MUTACAO, TAMANHO_TORNEIO, MASTER_SEED)

In [26]:
# Instância E-n23-k3.evrp
executar('E-n23-k3.evrp')

Iniciando 20 execuções do AG Avançado (Permutação) para a instância E-n23-k3.evrp
Semente Mestra do Experimento: 21
Número de Veículos Disponíveis: 1
Dimensão (N): 23
Número máximo de avaliações por execução: 575000
Tamanho da População: 100
Taxa de Mutação: 0.02
Crossover: Cycle Crossover
Mutação: Inversion Mutation
Seleção: Torneio (k=9)

AVISO: Este algoritmo NÃO considera capacidade de carga ou bateria nas rotas.

--- Execução 1/20 ---
  Melhor solução: [13, 12, 8, 22, 5, 6, 9, 10, 11, 14, 7, 2, 3, 4, 17, 16, 15, 18, 23, 21, 20, 19]
  Distância mínima encontrada nesta execução: 476.9451
  Rotas finais: [[1, 13, 12, 8, 22, 5, 6, 9, 10, 11, 14, 7, 2, 3, 4, 17, 16, 15, 18, 23, 21, 20, 19, 1]]
  Total de avaliações nesta execução: 575000

--- Execução 2/20 ---
  Melhor solução: [13, 8, 22, 5, 6, 9, 10, 11, 14, 12, 7, 2, 3, 4, 17, 16, 15, 18, 23, 21, 20, 19]
  Distância mínima encontrada nesta execução: 470.0586
  Rotas finais: [[1, 13, 8, 22, 5, 6, 9, 10, 11, 14, 12, 7, 2, 3, 4, 17, 16

In [27]:
# Instância E-n51-k5.evrp
executar('E-n51-k5.evrp')

Iniciando 20 execuções do AG Avançado (Permutação) para a instância E-n51-k5.evrp
Semente Mestra do Experimento: 21
Número de Veículos Disponíveis: 1
Dimensão (N): 51
Número máximo de avaliações por execução: 1275000
Tamanho da População: 100
Taxa de Mutação: 0.02
Crossover: Cycle Crossover
Mutação: Inversion Mutation
Seleção: Torneio (k=9)

AVISO: Este algoritmo NÃO considera capacidade de carga ou bateria nas rotas.

--- Execução 1/20 ---
  Melhor solução: [15, 26, 19, 14, 42, 41, 20, 43, 5, 48, 13, 18, 38, 45, 16, 46, 34, 40, 31, 11, 50, 10, 51, 35, 22, 30, 21, 36, 37, 4, 29, 32, 23, 3, 17, 39, 6, 12, 33, 2, 49, 9, 27, 8, 24, 44, 25, 7, 28, 47]
  Distância mínima encontrada nesta execução: 469.3422
  Rotas finais: [[1, 15, 26, 19, 14, 42, 41, 20, 43, 5, 48, 13, 18, 38, 45, 16, 46, 34, 40, 31, 11, 50, 10, 51, 35, 22, 30, 21, 36, 37, 4, 29, 32, 23, 3, 17, 39, 6, 12, 33, 2, 49, 9, 27, 8, 24, 44, 25, 7, 28, 47, 1]]
  Total de avaliações nesta execução: 1275000

--- Execução 2/20 ---
  M